Выставлю seed для тестирования используемых моделей:

In [ ]:
SEED = 3333

# 1. Предобработка данных

In [ ]:
import pandas as pd

df_train_base = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
df_test_base = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [ ]:
df_train_base.head()

Объединю тренировочный и тестовый датасет для удобства обработки и избавлюсь от очевидно лишних фич (id и имя):

In [ ]:
df_test_base['Transported'] = False

In [ ]:
df = pd.concat([df_train_base, df_test_base])
df = df.drop(columns=['PassengerId', 'Name'])

В описании датасета сказано: "Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard." => разделю фичу Cabin на 3 фичи Deck, Num, Side:

In [ ]:
df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand=True)
df['Num'] = pd.to_numeric(df['Num'])
df = df.drop(columns=['Cabin'])

In [ ]:
df.head()

В датасете есть пропуски. Есть вариант удалить объекты с пропусками, но я воспользуюсь KNNImputer, который использует алгоритм k ближайших соседей, для заполнения NaN значений у количественных фич. Для номинальных же добавлю новое значение - 'unknown'

In [ ]:
from sklearn.impute import KNNImputer

imp = KNNImputer()
impute_list = ['Age', 'VIP', 'Num', 'CryoSleep', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
rest_list = list(set(df.columns) - set(impute_list))
df_imputed = pd.DataFrame(imp.fit_transform(df[impute_list]), columns=impute_list)
df_rest = df[rest_list]
df = pd.concat([df_rest.reset_index(drop=True), df_imputed.reset_index(drop=True)], axis=1)

In [ ]:
df['Destination'] = df['Destination'].fillna('unknown')
df['Deck'] = df['Deck'].fillna('unknown')
df['Side'] = df['Side'].fillna('unknown')
df['HomePlanet'] = df['HomePlanet'].fillna('unknown')

Составлю матрицу корреляции для количественных признаков и увижу, что признаки слабо коррелируют => никакие удалять не нужно.

In [ ]:
import seaborn as sns

sns.heatmap(df[['Age', 'VIP', 'Num', 'CryoSleep', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].corr(), cmap='coolwarm')

Рассмотрю номинальные фичи и замечу, что уникальных значений довольно мало => можно воспользоваться One Hot Encoding вместо Label Encoding/Frequency Encoding.

In [ ]:
print(df['Destination'].unique())
print(df['Deck'].unique())
print(df['Side'].unique())
print(df['HomePlanet'].unique())

In [ ]:
df = pd.get_dummies(df, columns=['HomePlanet', 'Destination', 'Deck', 'Side'])

Проверю, что все NaN значения у объектов были убраны:

In [ ]:
df.isna().sum()

Разделю датасет обратно на тренировочный и тестовый:

In [ ]:
df_train, df_test = df[:df_train_base.shape[0]], df[df_train_base.shape[0]:].drop(columns=['Transported'])
x_train = df_train.drop(columns=['Transported'])
y_train = df_train['Transported']

# 2. Выбор и обучение модели

Для выбора модели напишу функцию, которая разделит тренировочный датасет в соотношении 80/20 и будет вычислять метрику accuracy на основе этого разделения. Протестирую модели, пока без тюнинга гиперпараметров (прописанный verbose нужен лишь для сокращения вывода, а max_iter в LogisticRegression, чтобы убрать назойливый warning -_-)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def test_model(train, test, model_type, **kwargs):
    x_train, x_test, y_train, y_test = train_test_split(train, test, random_state=SEED, test_size=0.2)    
    model = model_type(**kwargs)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    return accuracy_score(y_pred, y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

print(f'LogisticRegression: {test_model(x_train, y_train, LogisticRegression, verbose=0, max_iter=1000)}')
print(f'DecisionTreeClassifier: {test_model(x_train, y_train, DecisionTreeClassifier)}')
print(f'RandomForestClassifier: {test_model(x_train, y_train, RandomForestClassifier, verbose=0)}')
print(f'LGBMClassifier: {test_model(x_train, y_train, LGBMClassifier, verbose=-1)}')
print(f'XGBClassifier: {test_model(x_train, y_train, XGBClassifier)}')

Как можно увидеть LGBMClassifier получил лучший результат, осталось подобрать параметры, для этого воспользуюсь RandomizedSearchCV (optuna мне не понравилась + концептуально она делает то же самое)

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# import numpy as np

# model = LGBMClassifier(
#     n_jobs=1,
#     verbose=-1,
#     force_row_wise=True
# )

# param_dist = {
#     'n_estimators': np.arange(100, 1000),
#     'max_depth': np.arange(2, 10),
#     'num_leaves': np.arange(20, 200, 10),
# }

# random_search = RandomizedSearchCV(
#     estimator=model,
#     param_distributions=param_dist,
#     n_iter=1000,
#     cv=5,
#     n_jobs=-1,
#     scoring='accuracy',
#     random_state=SEED
# )

# random_search.fit(x_train, y_train)

# print(random_search.best_params_)
# print(random_search.best_score_)

Немного поменяю итоговые гиперпаметры вручную (а именно параметр max_depth) для достижения максимального результата в функции test_model. Обучу модель с данными гиперпараметрами и попытаюсь предсказать значения для тестового датасета у целевой фичи.

In [ ]:
print(f'LGBMClassifier: {test_model(x_train, y_train, LGBMClassifier, random_state=SEED, verbose=-1, num_leaves=170, n_estimators=176, max_depth=4)}')


In [ ]:
model = LGBMClassifier(random_state=SEED, verbose=-1, num_leaves=170, n_estimators=176, max_depth=4)
model.fit(x_train, y_train)
y_pred = model.predict(df_test)
submission = pd.DataFrame({'PassengerId': df_test_base.PassengerId, 'Transported': y_pred})
submission.to_csv('/kaggle/working/submission.csv', index=False)
submission.to_csv('./submission.csv', index=False)